In [4]:
import sys
import time

import ollama
import jericho
import re
import json
import random

from sklearn.metrics.pairwise import cosine_similarity

In [5]:
GAMES_DIR = "z-machine-games-master/jericho-game-suite"
game = 'zork1.z5'
env = jericho.FrotzEnv(f"{GAMES_DIR}/{game}")

In [6]:
initial_observation, info = env.reset()
print(initial_observation)

parts = initial_observation.split(' ')
middle = len(parts) // 2
first_half = ' '.join(parts[:middle+3])
second_half = ' '.join(parts[middle+3:])

print('first')
print(first_half)
print()
print('second')
print(second_half)


Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


first
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are

second
standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.




In [7]:
def embedding(text):
    embed_result = ollama.embed(model='nomic-embed-text:latest', input=text)
    # print(type(embed_result.embeddings))
    # print(embed_result.embeddings)
    return embed_result.embeddings

In [8]:
actual = embedding(initial_observation)
actual

[[0.003380086,
  0.028641177,
  -0.20731561,
  -0.0531142,
  0.040126424,
  0.022249332,
  1.5344145e-05,
  0.055960968,
  0.08155047,
  -0.03719124,
  -0.03514607,
  0.028711922,
  0.016595127,
  -0.010641888,
  0.017716078,
  -0.045954205,
  0.053810123,
  -0.070711,
  -0.022942051,
  0.018534083,
  0.009196966,
  0.00662489,
  -0.050868344,
  -0.015944717,
  0.08486651,
  0.0047782767,
  0.04918115,
  0.022196,
  0.00079265214,
  -0.028521271,
  -0.005742135,
  0.014803275,
  -0.024961503,
  0.00894201,
  -0.005235983,
  -0.0011901245,
  0.016886197,
  0.019853797,
  0.025256729,
  -0.026693264,
  0.06760423,
  0.036670394,
  -0.028068146,
  0.018736744,
  0.015599732,
  0.023915228,
  -0.031572953,
  0.03805489,
  0.081525646,
  0.035305116,
  0.049446065,
  8.945677e-05,
  0.028827835,
  -0.011557975,
  -0.023673614,
  -0.016673135,
  0.012287638,
  -0.0017163006,
  -0.011370117,
  0.025971413,
  0.09310577,
  0.093873,
  -0.0026437317,
  0.08064763,
  0.04936706,
  -0.012860028,


In [9]:
prompt = f'Below is the first half of the very first text prompt given by the text adventure game Zork. Please complete it exactly as it is given by Zork. Do not include anything except your completion.\n\n"{first_half}" (completion here...)'

In [10]:
def test_llama():
    llama_response = ollama.generate(model='llama3.2:3b', prompt=prompt).response
    rebuilt_llama = first_half + ' ' + llama_response
    # print(rebuilt_llama)
    llama_embed = embedding(rebuilt_llama)
    sim = cosine_similarity(llama_embed, actual)
    # print(sim)
    return {'text': rebuilt_llama, 'sim': sim}

In [11]:
def test_qwen():
    qwen_response = ollama.generate(model='qwen3', prompt=prompt, options={'num_ctx': 2048}).response
    rebuilt_qwen = first_half + ' ' + qwen_response
    # print(rebuilt_qwen)
    qwen_embed = embedding(rebuilt_qwen)
    sim = cosine_similarity(qwen_embed, actual)
    # print(sim)
    return {'text': rebuilt_qwen, 'sim': sim}

In [12]:
def test_gpt():
    gpt_response = ollama.generate(model='gpt-oss', prompt=prompt, options={'num_ctx': 2048}).response
    rebuilt_gpt = first_half + ' ' + gpt_response
    # print(rebuilt_gpt)
    gptoss_embed = embedding(rebuilt_gpt)
    sim = cosine_similarity(gptoss_embed, actual)
    # print(sim)
    return {'text': rebuilt_gpt, 'sim': sim}

In [86]:
llama_tests = [test_llama() for _ in range(30)]

In [ ]:
mean_llama = sum(x['sim'] for x in llama_tests) / 30
mean_llama

array([[27.97977343]])

In [ ]:
qwen_tests = [test_qwen() for _ in range(30)]

In [ ]:
mean_qwen = sum(x['sim'] for x in qwen_tests) / 30
mean_qwen

In [ ]:
gpt_tests = [test_gpt() for _ in range(30)]

In [ ]:
mean_gpt = sum(x['sim'] for x in gpt_tests) / 30
mean_gpt

In [39]:
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv();

In [40]:
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [47]:
without_copyright = '''West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.
'''
actual = embedding(without_copyright)

first_half = '''West of House
You are standing in an open field west of
'''

prompt = f'Below is the first half of the very first text prompt given by the text adventure game Zork. Please complete it exactly as it is given by Zork. Do not include anything except your completion.\n\n"{first_half}" (completion here...)'
response = client.responses.create(
        model="gpt-5.2",
        input=prompt
    )

In [48]:
reply = response.output_text
reply

'a white house, with a boarded front door.\nThere is a small mailbox here.'

In [49]:
rebuilt = first_half + ' ' + reply.split(':')[-1]
print(rebuilt)
cosine_similarity(embedding(rebuilt), actual)

West of House
You are standing in an open field west of
 a white house, with a boarded front door.
There is a small mailbox here.


array([[1.]])

ChatGPT will normally answer that it can't complete the copyrighted game text. About 1 in 5 tries, it gets it exactly or almost exactly